In [21]:
import pandas as pd
import numpy as np

database=pd.read_csv("../datasets/database_corregido.csv",encoding='UTF-8')

In [22]:
database.head()

,user_id,user_name,usersurname,email,password,year_birth,gender,mother_tongue,years_in,studies,working,support_type,expert,area,about_me,date,country,pic
0,1,ZpafJwb,IGLxqLcD,SidQXyFROW@gmail.com,FuDgmKsOA,1997,Femenino,"Frances, Chino",De 1 año a 2 años,Sin graduado escolar,Si,Emocional,No,Valle del Jarama,"Juegos, Baile, Musica",13/10/2021,NaN,NaN
1,2,vraVenl,pQgDlKpb,dmRMlEHSix@gmail.com,TcEDUaWhT,1983,No especificar,"Aleman, Chino",De 1 año a 2 años,Master,No especificar,Orientacion sobre temas legales,Si,Valle Bajo del Lozoya,"Lectura, Baile, Cine",12/07/2021,NaN,NaN
2,3,ZCogidr,pyqWoZEy,InOMyDalbC@gmail.com,YigDCDzGH,1998,Masculino,"Portugues, Ingles",Menos de 6 meses,Doctorado,No especificar,Orientacion laboral,No,Sierra del Rincon,"Viajar, Baile, Lectura",09/02/2021,NaN,NaN
3,4,vdTiFuY,DsyOuIJT,zXBFBgHyDk@gmail.com,juZCtMEMk,1968,Femenino,"Frances, Ucraniano",De 6 meses a 1 año,FP (Formacion Profesional),No especificar,Orientacion sobre tramites,Si,Valle Medio del Lozoya,"Deportes, Pintura, Baile",24/08/2022,NaN,NaN
4,5,sRnfLQq,sOAsLqnv,juObUXPbPT@gmail.com,IlpRnxgfA,1982,No especificar,"Ucraniano, Ingles",De 6 meses a 1 año,Sin graduado escolar,Si,Orientacion sobre temas legales,Si,Valle Alto del Lozoya,"Cocina, Juegos, Musica",21/04/2021,NaN,NaN


In [23]:
# Funcion que utiliza el id recibido para crear una lista con los datos del usuario y un string del tipo de apoyo

def lista_datos(x):
    # database_copia = database.drop(['user_name', 'usersurname', 'email', 'password', 'pic', 'country', 'date', 'studies'],axis=1)
    user=database.loc[database["user_id"] == x]
    tipo_apoyo = user.support_type[0]
    lista_user=user.values.flatten().tolist()
    return lista_user, tipo_apoyo

# Función que utiliza como argumento el string de tipo de apoyo y filtra para devolver un dataframe

def tipo_apoyo(apoyo):
    if apoyo == 'Orientacion sobre temas legales':
        df_temas_legales = database[(database['support_type']=='Orientacion sobre temas legales')]
        # df_temas_legales = df_temas_legales.drop(['user_name', 'usersurname', 'email', 'password', 'gender', 'studies', 'pic', 'country', 'date', 'about_me'],axis=1)
        df_temas_legales = df_temas_legales[df_temas_legales['expert']=='Si']

        return df_temas_legales

    elif apoyo == 'Orientacion sobre tramites':
        df_tramites = database[(database['support_type']=='Orientacion sobre tramites')]
        # df_tramites = df_tramites.drop(['user_name', 'usersurname', 'email', 'password', 'gender', 'studies', 'pic', 'country', 'date', 'about_me'],axis=1)
        df_tramites = df_tramites[df_tramites['expert']=='Si']

        return df_tramites

    elif apoyo == 'Orientacion laboral':
        df_tramites_laborales = database[(database['support_type']=='Orientacion laboral')]
        # df_tramites_laborales = df_tramites_laborales.drop(['user_name', 'usersurname', 'email', 'password', 'gender', 'years_in', 'about_me', 'pic', 'country', 'date'],axis=1)
        df_tramites_laborales = df_tramites_laborales[df_tramites_laborales['expert']=='Si']

        return df_tramites_laborales

    else:
        df_emocional = database[(database['support_type']=='Emocional')]
        # df_emocional = df_emocional.drop(['user_name', 'usersurname', 'email', 'password', 'pic', 'country', 'date', 'studies'],axis=1)
        df_emocional = df_emocional[df_emocional['expert']=='Si']

        return df_emocional

# Función que toma como argumentos la lista de datos del usuario y el dataframe de expertos segun el tipo de apoyo y devuelve un df que va para el modelo

def filtro_idioma(datos_usuario,df_support):
    if ',' in datos_usuario[df_support.columns.get_loc('mother_tongue')]:
        datos_usuario[df_support.columns.get_loc('mother_tongue')] = [datos_usuario[df_support.columns.get_loc('mother_tongue')]]
        div_idioma = datos_usuario[df_support.columns.get_loc('mother_tongue')][0].split(', ')
        df_model = df_support[(df_support.mother_tongue.str.contains(div_idioma[0])) | (df_support.mother_tongue.str.contains(div_idioma[1]))]

    else:
        df_model = df_support[(df_support.mother_tongue.str.contains(datos_usuario[df_support.columns.get_loc('mother_tongue')]))]

    return df_model


# Función que ejecuta todas las funciones anteriores para poder realizarlo en una sola vez

def automatizacion(id):
    
    datos_usuario, apoyo = lista_datos(id)

    df_support = tipo_apoyo(apoyo=apoyo)

    df_salida = filtro_idioma(datos_usuario, df_support)
    return df_salida


In [31]:
id = 1
para_modelo = automatizacion(id=id)